In [1]:
import pyecharts
import pandas as pd
# 使用pyecharts自带的config来申明一下使用pyecharts的运行环境
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB


import pyecharts.options as opts

In [6]:
df = pd.read_excel("D:\\teaching\\New Media Analysis and User Experience\\survey\\inm.xlsx")

# 数据清洗

In [7]:
def gender(series):
    if series == 1:
        return '男性'
    else:
        return '女性'

def hukou(series):
    if series == 1:
        return '城镇'
    else:
        return '非城镇'

def location (series):
    if series == 1:
        return "非广东地区"
    elif series == 2:
        return "珠三角地区"
    elif series == 3:
        return "粤东"
    elif series == 4:
        return "粤西"
    else:
        return "粤北"

def like(series):
    if series == 1:
        return '喜欢'
    else:
        return '不喜欢'

def impr(series):
    if series == 1:
        return '有提升空间'
    else:
        return '挺好的'


In [8]:
df.gender = df.gender.apply(gender).astype('category')
# df.gender.value_counts()

df.hukou = df.hukou.apply(hukou).astype('category')
# df.hukou.value_counts()

df.location = df.location.apply(location).astype('category')
# df.location.value_counts()

In [9]:
# df.filter(regex = "^l_.*")

# 直方图

In [10]:
tb_eff = df.effort_overall.value_counts().rename_axis('unique_values').reset_index(name='counts').sort_values("unique_values")
tb_eff

,unique_values,counts
4,1,1
2,2,10
0,3,44
1,4,24
3,5,1


In [11]:
from pyecharts.charts import Bar

bar = Bar()
bar.add_xaxis(list(tb_eff.unique_values))
bar.add_yaxis("effort_overall", list(tb_eff.counts))
# render 会生成本地 HTML 文件，默认会在当前目录生成 render.html 文件
# 也可以传入路径参数，如 bar.render("mycharts.html")
# bar.load_javascript()
# bar.render_notebook()
bar.render("直方图.html")

'D:\\teaching\\New Media Analysis and User Experience\\survey\\直方图.html'

# 饼状图

## 性别比例

In [12]:
tb_gender = df.gender.value_counts().rename_axis('gender').reset_index(name='counts')

tb_gender


,gender,counts
0,女性,64
1,男性,16


In [14]:
import pyecharts.options as opts
from pyecharts.charts import Pie

"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.baidu.com/examples/editor.html?c=pie-doughnut

目前无法实现的功能:

1、迷之颜色映射的问题
"""

x_data = tb_gender.gender.to_list()
y_data = tb_gender.counts.to_list()
data_pair = [list(z) for z in zip(x_data, y_data)]
data_pair.sort(key=lambda x: x[1])

(
    Pie(init_opts=opts.InitOpts(width="1600px", height="800px", bg_color="#2c343c"))
    .add(
        series_name="男女比例",
        data_pair=data_pair,
        rosetype="radius",
        radius="55%",
        center=["50%", "50%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="网新学子性别比例饼状图",
            pos_left="center",
            pos_top="20",
            title_textstyle_opts=opts.TextStyleOpts(color="#fff"),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
    )
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(color="rgba(255, 255, 255, 0.3)"),
    )
#     .render_notebook()
      .render("饼图.html")
)


'D:\\teaching\\New Media Analysis and User Experience\\survey\\饼图.html'

## 生源地

In [17]:
tb_location = df.location.value_counts().rename_axis('location').reset_index(name='counts')
tb_location

,location,counts
0,珠三角地区,40
1,粤东,17
2,粤西,10
3,非广东地区,7
4,粤北,6


In [18]:
import pyecharts.options as opts
from pyecharts.charts import Pie

"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.baidu.com/examples/editor.html?c=pie-doughnut

目前无法实现的功能:

1、饼状图中间的图例名称暂时无法显示
"""

x_data = tb_location.location.to_list()
y_data = tb_location.counts.to_list()

(
    Pie(init_opts=opts.InitOpts(width="1600px", height="1000px"))
    .add(
        series_name="生源地",
        data_pair=[list(z) for z in zip(x_data, y_data)],
        radius=["50%", "70%"],
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(legend_opts=opts.LegendOpts(pos_left="legft", orient="vertical"))
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        # label_opts=opts.LabelOpts(formatter="{b}: {c}")
    )
#     .render_notebook()
      .render("饼图-生源地.html")
)


'D:\\teaching\\New Media Analysis and User Experience\\survey\\饼图-生源地.html'

# 箱型图

In [19]:
city_ranking = df[df.hukou == "城镇"].ranking.to_list()
rural_ranking = df[df.hukou == "非城镇"].ranking.to_list()

In [20]:
from pyecharts import options as opts
from pyecharts.charts import Boxplot

v1 = [city_ranking,rural_ranking]

c = Boxplot()
c.add_xaxis(["城镇", "非城镇"])
c.add_yaxis("户口",c.prepare_data(v1))
c.set_global_opts(title_opts=opts.TitleOpts(title="户口与学业成绩的箱型图"))


In [21]:
# c.render_notebook()
c.render("箱型图.html")

'D:\\teaching\\New Media Analysis and User Experience\\survey\\箱型图.html'

## 生源地与专业满意度

In [22]:
fei_sati = df[df.location == "非广东地区"].sati_overall.to_list()
pearl_sati = df[df.location == "珠三角地区"].sati_overall.to_list()
north_sati = df[df.location == "粤北"].sati_overall.to_list()
west_sati = df[df.location == "粤西"].sati_overall.to_list()
east_sati = df[df.location == "粤东"].sati_overall.to_list()

In [23]:
v2 = [fei_sati,pearl_sati,north_sati,west_sati,east_sati]

c = Boxplot()
c.add_xaxis(["非广东地区 ","珠三角地区 ","粤东 ","粤西 ","粤北"])
c.add_yaxis("生源地",c.prepare_data(v2))
c.set_global_opts(title_opts=opts.TitleOpts(title="生源地与专业满意度的箱型图"))

In [24]:
# c.render_notebook()
c.render("箱型图-生源地与专业满意度.html")

'D:\\teaching\\New Media Analysis and User Experience\\survey\\箱型图-生源地与专业满意度.html'

# 雷达图

In [26]:
cap_mean = df.filter(regex = "cap_*").mean().to_list()
cap_mean

[2.0, 3.3, 3.3125, 3.2875, 3.3, 3.1]

In [27]:
import pyecharts.options as opts
from pyecharts.charts import Radar

"""
Gallery 使用 pyecharts 1.1.0
参考地址: https://echarts.baidu.com/examples/editor.html?c=radar

目前无法实现的功能:

1、雷达图周围的图例的 textStyle 暂时无法设置背景颜色
"""
v1 = [cap_mean]

(
    Radar(init_opts=opts.InitOpts(width="1280px", height="720px", bg_color="#CCCCCC"))
    .add_schema(
        schema=[
            opts.RadarIndicatorItem(name="编程能力", max_=5),
            opts.RadarIndicatorItem(name="设计能力", max_=5),
            opts.RadarIndicatorItem(name="运营能力", max_=5),
            opts.RadarIndicatorItem(name="产品能力", max_=5),
            opts.RadarIndicatorItem(name="文案能力", max_=5),
            opts.RadarIndicatorItem(name="营销能力", max_=5),
        ],
        splitarea_opt=opts.SplitAreaOpts(
            is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
        ),
        textstyle_opts=opts.TextStyleOpts(color="#fff"),
    )
    .add(
        series_name="专业能力",
        data=v1,
        linestyle_opts=opts.LineStyleOpts(color="#CD0000"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="网新学子专业能力雷达图"), legend_opts=opts.LegendOpts()
    )
#     .render_notebook()
      .render("雷达图.html")
)

'D:\\teaching\\New Media Analysis and User Experience\\survey\\雷达图.html'

# 堆叠直方图

In [28]:
crosstab = pd.crosstab(df.gender,df.l_code)
crosstab

l_code,0,1
gender,,
女性,56,8
男性,11,5


In [29]:
female = crosstab.iloc[0,:].to_list()
male = crosstab.iloc[1,:].to_list()

In [30]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker

c = (
    Bar()
    .add_xaxis(["不喜欢编程类课程","喜欢编程类课程"])
    .add_yaxis("女生", female, stack="stack1")
    .add_yaxis("男生", male, stack="stack1")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="性别与喜不喜欢编程类课程的关系的堆叠图"))
)

In [31]:
# c.render_notebook()
c.render("堆叠图.html")

'D:\\teaching\\New Media Analysis and User Experience\\survey\\堆叠图.html'